In [1]:
import cv2
import math
import copy
from scipy import ndimage
import solver
from solver import *


import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'
import keyboard

In [2]:
import utils
from utils import *

In [3]:
# Init grid to store Sudoku Board digits
SIZE = 9
grid = []
for i in range(SIZE):
    row = []
    for j in range(SIZE):
        row.append(0)
    grid.append(row)
   

In [4]:
import cv2
import operator

margin = 10
case = 28 + 2*margin
perspective_size = 9*case

flag = 0
ans = 0

cap = cv2.VideoCapture(0)

old_sudoku = None

while(True):
    ret, frame = cap.read()
    p_frame = frame.copy()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
     
    gray = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                cv2.THRESH_BINARY,11,2)
    
    contour,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contour_grille = None
    maxArea = 0
    for c in contour:
        area = cv2.contourArea(c)
        if area > 30000:
            peri = cv2.arcLength(c, True)
            polygone = cv2.approxPolyDP(c, 0.01 * peri, True)
            if area > maxArea and len(polygone) == 4:
                contour_grille = polygone
                maxArea = area
    if contour_grille is not None:
        cv2.drawContours(frame, [contour_grille], 0, (0, 255, 0), 2)
        if keyboard.is_pressed('o'):
            print('Contor detected')

            corners = get_corners_from_contours(contour_grille, 4)

            pts1=np.float32(sort_corners(corners))
            (tl, tr, br, bl) = pts1
            width_A = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
            width_B = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))

            # the height of our Sudoku board
            height_A = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
            height_B = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

            # take the maximum of the width and height values to reach
            # our final dimensions
            max_width = max(int(width_A), int(width_B))
            max_height = max(int(height_A), int(height_B))

            # construct our destination points which will be used to
            # map the screen to a top-down, "birds eye" view
            dst = np.array([
           [0, 0],
            [max_width - 1, 0],
          [max_width - 1, max_height - 1],
            [0, max_height - 1]], dtype = "float32")

            # calculate the perspective transform matrix and warp
            # the perspective to grab the screen
            perspective_transformed_matrix = cv2.getPerspectiveTransform(pts1, dst)
            warp = cv2.warpPerspective(frame, perspective_transformed_matrix, (max_width, max_height))
            orginal_warp = np.copy(warp)
            print('Image Extracted detected')

    cv2.imshow('f',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()     

Contor detected
Image Extracted detected
Contor detected
Image Extracted detected
Contor detected
Image Extracted detected
Contor detected
Image Extracted detected


In [5]:
cv2.imshow('p',warp)
cv2.waitKey(0)

-1

In [6]:

p_window = cv2.cvtColor(warp, cv2.COLOR_BGR2GRAY)
p_window = cv2.GaussianBlur(p_window, (5,5), 0)
p_window = cv2.adaptiveThreshold(p_window, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11, 2)
p= cv2.bitwise_not(p_window)
_,p = cv2.threshold(p, 200, 255, cv2.THRESH_BINARY)

In [7]:
cv2.imshow('p',p)
cv2.waitKey(0)

-1

In [8]:
height =p.shape[0] // 9
width =p.shape[1] // 9

offset_width = math.floor(width / 10)    # Offset is used to get rid of the boundaries
offset_height = math.floor(height / 10)

In [11]:
for i in range(SIZE):
        for j in range(SIZE):
                y2min = width*i+offset_width
                y2max = width*(i+1)-offset_width
                x2min = height*j+offset_height
                x2max = height*(j+1)-offset_height

                # Crop with offset (We don't want to include the boundaries)
                crop_image = p[y2min:y2max,x2min:x2max]        

                # There are still some boundary lines left though
                # => Remove all black lines near the edges
                # ratio = 0.6 => If 60% pixels are black, remove
                # Notice as soon as we reach a line which is not a black line, the while loop stops
                ratio = 0.6        
                # Top
                while np.sum(crop_image[0]) <= (1-ratio) * crop_image.shape[1] * 255:
                    crop_image = crop_image[1:]
                # Bottom
                while np.sum(crop_image[:,-1]) <= (1-ratio) * crop_image.shape[1] * 255:
                    crop_image = np.delete(crop_image, -1, 1)
                # Left
                while np.sum(crop_image[:,0]) <= (1-ratio) * crop_image.shape[0] * 255:
                    crop_image = np.delete(crop_image, 0, 1)
                # Right
                while np.sum(crop_image[-1]) <= (1-ratio) * crop_image.shape[0] * 255:
                    crop_image = crop_image[:-1]    

                # Take the largestConnectedComponent (The digit), and remove all noises
                crop_image = cv2.bitwise_not(crop_image)
                crop_image = largest_connected_component(crop_image)

                digit_pic_size=30
                crop_image= cv2.resize(crop_image,(digit_pic_size,digit_pic_size))

                # If this is a white cell, set grid[i][j] to 0 and continue on the next image:
                # Criteria 1 for detecting white cell:
                # Has too little black pixels
                if crop_image.sum() >= digit_pic_size**2*255 - digit_pic_size * 1 * 255:
                    grid[i][j] = 0
                    continue


            # Criteria 2 for detecting white cell
                # Huge white area in the center
                center_width = crop_image.shape[1] // 2
                center_height = crop_image.shape[0] // 2
                x_start = center_height // 2
                x_end = center_height // 2 + center_height
                y_start = center_width // 2
                y_end = center_width // 2 + center_width
                center_region = crop_image[x_start:x_end, y_start:y_end]

                if center_region.sum() >= center_width * center_height * 255 - 255:
                    grid[i][j] = 0
                    continue

                # Now we are quite certain that this crop_image contains a number

                # Store the number of rows and cols
                rows, cols = crop_image.shape 
                 # Apply Binary Threshold to make digits more clear
                _, crop_image = cv2.threshold(crop_image, 200, 255, cv2.THRESH_BINARY)
                crop_image = crop_image.astype(np.uint8)


                crop_image = cv2.bitwise_not(crop_image)
                shift_x, shift_y = get_best_shift(crop_image)
                shifted = shift(crop_image,shift_x,shift_y)
                crop_image = shifted
                cv2.imwrite('Digit/'+str(i)+str(j)+'.png',crop_image)

                crop_image = cv2.bitwise_not(crop_image)  

                 # Recognize digits
                prediction = pytesseract.image_to_string(crop_image, lang='eng', config='--psm 6')
                grid[i][j] =prediction
            
                if type(grid[i][j]) is not int:
                    grid[i][j]=int(grid[i][j].replace('\n\x0c', ""))

ValueError: invalid literal for int() with base 10: '\x0c'

In [10]:
def get_best_shift(img):
    cy, cx = ndimage.measurements.center_of_mass(img)
    rows, cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)
    return shiftx, shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

In [19]:
grid

[[5, 3, 0, 0, 7, 0, 0, 0, 0],
 [6, 0, 0, 1, 9, 5, 0, 0, 0],
 [0, 9, 8, 0, 0, 0, 0, 6, 0],
 [8, 0, 0, 0, 6, 0, 0, 0, 3],
 [4, 0, 0, 8, 0, 3, 0, 0, 1],
 [7, 0, 0, 0, 2, 0, 0, 0, 6],
 [0, 6, 0, 0, 0, 0, 2, 8, 0],
 [0, 0, 0, 4, 1, 9, 0, 0, 5],
 [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [20]:
predicted_digits=grid.copy()

In [21]:
solver.solve_sudoku(predicted_digits)

In [22]:
predicted_digits

[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]

In [24]:
result=write_solution_on_image(warp,predicted_digits,grid)
cv2.imshow("Result", result)

In [30]:
SIZE = 9
width = warp.shape[1] // 9
height = warp.shape[0] // 9
for i in range(SIZE):
    for j in range(SIZE):
        if grid[i][j] == 0:
            text = str(predicted_digits[i][j])
            off_set_x = width // 15
            off_set_y = height // 15
            font = cv2.FONT_HERSHEY_SIMPLEX
            (text_height, text_width), baseLine = cv2.getTextSize(text, font, fontScale=1, thickness=3)
            marginX = math.floor(width / 7)
            marginY = math.floor(height / 7)

            font_scale = 0.6 * min(width, height) / max(text_height, text_width)
            text_height *= font_scale
            text_width *= font_scale
            bottom_left_corner_x = width*j + math.floor((width - text_width) / 2) + off_set_x
            bottom_left_corner_y = height*(i+1) - math.floor((height - text_height) / 2) + off_set_y
            image = cv2.putText(original_warp, text, (bottom_left_corner_x, bottom_left_corner_y), 
                                                  font, font_scale, (0,255,0), thickness=3, lineType=cv2.LINE_AA)
        else:
            continue

In [32]:
cv2.imshow('result',image)
cv2.waitKey(0)

-1